In [22]:
import pandas as pd
import nltk as n
import itertools as i
import numpy as np

In [8]:
data = pd.read_csv('spam.csv',skiprows=1, names=['type','text', 'c3', 'c4', 'c5'], delimiter=',', encoding='ISO-8859-1')

In [9]:
data.drop(['c3','c4','c5'],axis=1,inplace=True)

In [114]:
smoother = 0.00001

In [89]:
data

,type,text,split
0,ham,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point,, crazy.., available..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar..., joking, wif, u, oni...]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor..., u, c, already..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, i, don't, think, he, goes, to, usf,, he,..."
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,..."
5568,ham,Will Ì_ b going to esplanade fr home?,"[will, ì_, b, going, to, esplanade, fr, home?]"
5569,ham,"Pity, * was in mood for that. So...any other s...","[pity,, *, was, in, mood, for, that., so...any..."
5570,ham,The guy did some bitching but I acted like i'd...,"[the, guy, did, some, bitching, but, i, acted,..."


In [90]:
set(data.type)

{'ham', 'spam'}

In [91]:
data['split'] = data.text.apply(lambda x: x.lower().split())

In [92]:
data[:5]

,type,text,split
0,ham,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point,, crazy.., available..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar..., joking, wif, u, oni...]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor..., u, c, already..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, i, don't, think, he, goes, to, usf,, he,..."


In [93]:
vocabulary = [w for s in data.split for w in s ]

In [94]:
len(vocabulary)

86335

In [119]:
len(set(vocabulary))

vocabulary[:100]

['go',
 'until',
 'jurong',
 'point,',
 'crazy..',
 'available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet...',
 'cine',
 'there',
 'got',
 'amore',
 'wat...',
 'ok',
 'lar...',
 'joking',
 'wif',
 'u',
 'oni...',
 'free',
 'entry',
 'in',
 '2',
 'a',
 'wkly',
 'comp',
 'to',
 'win',
 'fa',
 'cup',
 'final',
 'tkts',
 '21st',
 'may',
 '2005.',
 'text',
 'fa',
 'to',
 '87121',
 'to',
 'receive',
 'entry',
 'question(std',
 'txt',
 "rate)t&c's",
 'apply',
 "08452810075over18's",
 'u',
 'dun',
 'say',
 'so',
 'early',
 'hor...',
 'u',
 'c',
 'already',
 'then',
 'say...',
 'nah',
 'i',
 "don't",
 'think',
 'he',
 'goes',
 'to',
 'usf,',
 'he',
 'lives',
 'around',
 'here',
 'though',
 'freemsg',
 'hey',
 'there',
 'darling',
 "it's",
 'been',
 '3',
 "week's",
 'now',
 'and',
 'no',
 'word',
 'back!',
 "i'd",
 'like',
 'some',
 'fun',
 'you',
 'up',
 'for',
 'it',
 'still?']

In [96]:
test = data.sample(frac=0.1)
train = data[~data.index.isin(test.index)]

### Estimate P(T)

In [97]:
from collections import Counter as ctr

p_t_estimate = ctr(train.type)
p_t_total = len(train)

In [98]:
p_t_estimate['ham']

4335

In [99]:
def Pt(T):
    return p_t_estimate[T] / p_t_total

In [100]:
Pt('ham'), Pt('spam')

(0.864406779661017, 0.13559322033898305)

### Estimate P(W)

In [101]:
words = [w for s in train.split for w in s]
p_w_estimate = ctr(words)
p_w_total = len(words)

In [102]:
def Pw(W):
    if W not in p_w_estimate: return smoother
    return p_w_estimate[W] / p_w_total

In [103]:
Pw('the')

0.015334512911249583

In [104]:
np.sum([Pw(w) for w in set(words)])

1.0

### Estimate P(W|T)

In [105]:
p_w_t_estimate = {}
p_w_t_totals = {}

for t in set(train.type):
    sub_frame = train[train.type == t]
    sub_words = [w for s in sub_frame.split for w in s]
    p_w_t_estimate[t] = ctr(sub_words)
    p_w_t_totals[t] = len(sub_words)
    

In [106]:
def Pwt(W, T):
    if W not in p_w_t_estimate[T]: return smoother
    return p_w_t_estimate[T][W] / p_w_t_totals[T]

### Bayes' Rule

In [107]:
def Ptw(T, W):
    return Pwt(W, T) * Pt(T) / Pw(W)

In [108]:
def Pe(E):
    result = {}
    for t in set(train.type):
        result[t] = np.prod([Ptw(t, word) for word in E])
    return result

In [109]:
Pe(['the', 'river', 'is', 'long'])

{'ham': 0.9873764176834305, 'spam': 2.0869049307631213}

In [110]:
len(test)

557

In [115]:
test['result'] = test.split.apply(Pe)

In [116]:
test['top'] = test.result.apply(lambda x:max(x, key=x.get))

In [118]:
sum(test.type == test.top) / len(test)

0.9245960502692998